# Lesson 13 - Solutions

In [ ]:
import pandas as pd
import json

data = pd.read_csv("../../../../data/stumbleupon.tsv", sep = "\t")
data["title"] = data.boilerplate.map(lambda x: json.loads(x).get("title", ""))
data["body"] = data.boilerplate.map(lambda x: json.loads(x).get("body", ""))
data.head()

## Predicting "Greenness" of Content

This data set comes from [stumbleupon](https://www.stumbleupon.com/), a web page recommender

A description of the columns is below

FieldName|Type|Description
---------|----|-----------
url|string|Url of the webpage to be classified
title|string|Title of the article
body|string|Body text of article
urlid|integer| StumbleUpon's unique identifier for each url
boilerplate|json|Boilerplate text
alchemy_category|string|Alchemy category (per the publicly available Alchemy API found at [www.alchemyapi.com](www.alchemyapi.com))
alchemy_category_score|double|Alchemy category score (per the publicly available Alchemy API found at [www.alchemyapi.com](www.alchemyapi.com))
avglinksize| double|Average number of words in each link
commonlinkratio_1|double|# of links sharing at least 1 word with 1 other links / # of links
commonlinkratio_2|double|# of links sharing at least 1 word with 2 other links / # of links
commonlinkratio_3|double|# of links sharing at least 1 word with 3 other links / # of links
commonlinkratio_4|double|# of links sharing at least 1 word with 4 other links / # of links
compression_ratio|double|Compression achieved on this page via gzip (measure of redundancy)
embed_ratio|double|Count of number of embed usage
frameBased|integer (0 or 1)|A page is frame-based (1) if it has no body markup but have a frameset markup
frameTagRatio|double|Ratio of iframe markups over total number of markups
hasDomainLink|integer (0 or 1)|True (1) if it contains an `<a>` with an url with domain
html_ratio|double|Ratio of `<html>` tags vs text in the page
image_ratio|double|Ratio of `<img>` tags vs text in the page
is_news|integer (0 or 1) | True (1) if StumbleUpon's news classifier determines that this webpage is news
lengthyLinkDomain| integer (0 or 1)|True (1) if at least 3 `<a>`'s text contains more than 30 alphanumeric characters
linkwordscore|double|Percentage of words on the page that are in hyperlink's text
news_front_page| integer (0 or 1)|True (1) if StumbleUpon's news classifier determines that this webpage is front-page news
non_markup_alphanum_characters|integer| Page's text's number of alphanumeric characters
numberOfLinks|integer Number of <a>|markups
numwords_in_url| double|Number of words in url
parametrizedLinkRatio|double|A link is parametrised if it is url contains parameters or has an attached onClick event
spelling_errors_ratio|double|Ratio of words not found in wiki (considered to be a spelling mistake)
label|integer (0 or 1)|User-determined label. Either evergreen (1) or non-evergreen (0); available for train.tsv only

### Let's try extracting some of the text content
Create a feature for the title containing "recipe". Is the % of evegreen websites higher or lower on pages that have recipe in the the title?

In [ ]:
# Option 1: Create a function to check for this
def has_recipe(text_in):
    try:
        if "recipe" in str(text_in).lower():
            return 1
        else:
            return 0
    except: 
        return 0
        
data["recipe"] = data["title"].map(has_recipe)

# Option 2: lambda functions
# data["recipe"] = data["title"].map(lambda t: 1 if "recipe" in str(t).lower() else 0)

# Option 3: string functions
# data["recipe"] = data["title"].str.contains("recipe")

### Demonstration: Use of the Count Vectoriser

In [ ]:
titles = data["title"].fillna("")
from sklearn.feature_extraction.text import CountVectorizer

vectoriser = CountVectorizer(max_features = 1000,
                             ngram_range  = (1, 2),
                             stop_words   = "english",
                             binary       = True)

# Use `fit` to learn the vocabulary of the titles
vectoriser.fit(titles)

# Use `transform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = vectoriser.transform(titles)

### Demonstration: Build a random forest model to predict evergreeness of a website using the title features

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 20)

# Use `fit` to learn the vocabulary of the titles
vectoriser.fit(titles)

# Use `transform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = vectoriser.transform(titles).toarray()
y = data["label"]

from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, scoring = "roc_auc")
print("CV AUC {}, Average AUC {}".format(scores, scores.mean()))

### Exercise: Build a random forest model to predict evergreeness of a website using the title features and quantitative features

In [ ]:
# Use `transform` to generate the sample X word matrix - one column per feature (word or n-grams)
X_text_features = vectoriser.transform(titles)

# Identify the features you want from the original data set
other_features_columns = ["html_ratio", "image_ratio"]
other_features = data[other_features_columns]

# Stack them horizontally together
# This takes all of the word/n-gram columns and appends on two more columns for `html_ratio` and `image_ratio`
from scipy.sparse import hstack
X = hstack((X_text_features, other_features)).toarray()

scores = cross_val_score(model, X, y, scoring = "roc_auc")
print("CV AUC {}, Average AUC {}".format(scores, scores.mean()))

# What features of these are most important?
model.fit(X, y)

all_feature_names = vectoriser.get_feature_names() + other_features_columns
feature_importances = pd.DataFrame({"Features" : all_feature_names, "Importance Score": model.feature_importances_})
feature_importances.sort_values("Importance Score", ascending = False).head()

### Exercise: Build a random forest model to predict evergreeness of a website using the body features

In [ ]:
body_text = data["body"].fillna("")

# Use `fit` to learn the vocabulary
vectoriser.fit(body_text)

# Use `transform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = vectoriser.transform(body_text).toarray()

scores = cross_val_score(model, X, y, scoring = "roc_auc")
print("CV AUC {}, Average AUC {}".format(scores, scores.mean()))

### Exercise: Use `TfIdfVectorizer` instead of `CountVectorizer` - is this an improvement?

In [ ]:
titles = data["title"].fillna("")
from sklearn.feature_extraction.text import TfidfVectorizer

vectoriser = TfidfVectorizer(max_features = 1000,
                             ngram_range  = (1, 2),
                             stop_words   = "english")

# Use `fit` to learn the vocabulary
vectoriser.fit(body_text)

# Use `transform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = vectoriser.transform(body_text).toarray()

scores = cross_val_score(model, X, y, scoring = "roc_auc")
print("CV AUC {}, Average AUC {}".format(scores, scores.mean()))